# This notebook implement a baseline model in linear regression 

## Import and prepare dataset
Load the dataset out of .pkl file we preapared ind file "DataPreparation.ipynb".

In [11]:
# import all needed libs
import pandas as pd
import statsmodels.formula.api as smf

# read training data into datadframe
train_set = pd.read_pickle('../train_val_test_data/train_set.pkl')

## Try the first version of the model

In [12]:
# import all needed libs (if needed pip install)
import statsmodels.formula.api as smf


mod = smf.ols('Umsatz ~ Temperatur + monthly_mean_temp_diff + C(Warengruppe)', data=train_set).fit()

print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.695
Model:                            OLS   Adj. R-squared:                  0.695
Method:                 Least Squares   F-statistic:                     2440.
Date:                Sat, 01 Jun 2024   Prob (F-statistic):               0.00
Time:                        22:57:26   Log-Likelihood:                -43575.
No. Observations:                7487   AIC:                         8.717e+04
Df Residuals:                    7479   BIC:                         8.722e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 59

## Make predictions based on model above 

In [17]:
# load testset
test_set = pd.read_pickle('../train_val_test_data/test_set.pkl')

# calculate predictions for later upload 
test_set['Umsatz'] = mod.predict(test_set)

test_set.head()

# reduce to id and Umsatz columns 
submission_set = test_set[['id','Umsatz']]

# store the submission data
submission_set.to_csv('../prediction_data/submission.csv')
